In [76]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [78]:
import pandas as pd
allLabeledTweets = pd.read_csv('./../Naive_Bayes/tweets/allLabeledTweets.csv')
allLabeledTweets.head()

,message,id,tweetId,createdAt,language,inReplyToStatusId,inReplyToUserId,inReplyToScreenName,userId,userName,...,retweetedId,retweetCount,label,message_lowercase,clean_message,clean_message_no_punct,clean_message_no_stopwords_from_list,clean_message_no_punct_no_stopwords_from_list,clean_message_no_punct_no_freq_stopwords,clean_message_no_freq_stopwords
0,Tiek vērtēti trīs potenciālie airBaltic invest...,1478404,925617390523732000,2017-11-01T08:55:57,lv,NaN,NaN,NaN,24855060,Dienas Bizness,...,NaN,0,0,tiek vērtēti trīs potenciālie airbaltic invest...,tiek vērtēti trīs potenciālie airbaltic invest...,tiek vērtēti trīs potenciālie airbaltic invest...,vērtēti trīs potenciālie airbaltic investori U...,vērtēti trīs potenciālie airbaltic investori U...,tiek vērtēti trīs potenciālie airbaltic invest...,tiek vērtēti trīs potenciālie airbaltic invest...
1,"Vakardien, vēlu vakarā, ar svinīgu pasākumu ti...",1486476,924948828318511100,2017-10-30T12:39:20,lv,NaN,NaN,NaN,44849531,RIGA I Airport,...,NaN,0,0,"vakardien, vēlu vakarā, ar svinīgu pasākumu ti...","vakardien, vēlu vakarā, ar svinīgu pasākumu ti...",vakardien vēlu vakarā ar svinīgu pasākumu ti...,"vakardien , vēlu vakarā , svinīgu pasākumu atk...",vakardien vēlu vakarā svinīgu pasākumu atklāts...,vakardien vēlu vakarā svinīgu pasākumu tika at...,"vakardien , vēlu vakarā , svinīgu pasākumu tik..."
2,"Es ļoti ceru,ka potenciālie Air Baltic investo...",1488297,925794560420311000,2017-11-01T20:39:58,lv,NaN,NaN,NaN,218804015,Artis Pabriks,...,NaN,9,2,"es ļoti ceru,ka potenciālie air baltic investo...","es ļoti ceru,ka potenciālie air baltic investo...",es ļoti ceru ka potenciālie air baltic investo...,"es ļoti ceru , potenciālie air baltic investor...",es ļoti ceru potenciālie air baltic investori ...,ļoti ceru potenciālie air baltic investori nav...,"ļoti ceru , potenciālie air baltic investori n..."
3,Ar kritisko domāšanu gan joprojām bēdīgi. Piln...,1489818,925045141018218500,2017-10-30T19:02:02,lv,NaN,NaN,NaN,21071644,Andris Rubīns,...,NaN,3,2,ar kritisko domāšanu gan joprojām bēdīgi. piln...,ar kritisko domāšanu gan joprojām bēdīgi. piln...,ar kritisko domāšanu gan joprojām bēdīgi piln...,kritisko domāšanu joprojām bēdīgi . pilns face...,kritisko domāšanu joprojām bēdīgi pilns facebo...,kritisko domāšanu joprojām bēdīgi pilns facebo...,kritisko domāšanu joprojām bēdīgi . pilns face...
4,Latvijas lidsabiedrība airBaltic sadarbībā ar ...,1490250,925006964135952400,2017-10-30T16:30:20,lv,NaN,NaN,NaN,131562098,Latviesi.com,...,NaN,0,0,latvijas lidsabiedrība airbaltic sadarbībā ar ...,latvijas lidsabiedrība airbaltic sadarbībā ar ...,latvijas lidsabiedrība airbaltic sadarbībā ar ...,latvijas lidsabiedrība airbaltic sadarbībā apv...,latvijas lidsabiedrība airbaltic sadarbībā apv...,latvijas lidsabiedrība airbaltic sadarbībā apv...,latvijas lidsabiedrība airbaltic sadarbībā apv...


In [79]:
allLabeledTweets["label"].value_counts()

0    253
2    136
1     87
Name: label, dtype: int64

In [110]:
from sklearn import model_selection

train_df, test_df = model_selection.train_test_split(allLabeledTweets, test_size=0.1)
train_df.shape, test_df.shape

import numpy as np

features = allLabeledTweets.loc[:,"message_lowercase"].values
labels = allLabeledTweets.loc[:,"label"].values


csv_features = tf.data.experimental.make_csv_dataset(
    './../Naive_Bayes/tweets/allLabeledTweets.csv',
    batch_size=400,
    select_columns=['message_lowercase', 'label'],    
    label_name='label',
    num_epochs=1,
    ignore_errors=True,)

csv_labels = tf.data.experimental.make_csv_dataset(
    './../Naive_Bayes/tweets/allLabeledTweets.csv',
    batch_size=400,
    select_columns=['label'],    
    label_name='label',
    num_epochs=1,
    ignore_errors=True,)

# for batch, label in csv_ds.take(1):
#     for key, value in batch.items():
#         print(f"{key:20s}: {value}")
#         print()
#         print(f"{'label':20s}: {label}")

In [97]:
train_neutral = train_df[train_df["label"] == 0]
train_positive = train_df[train_df["label"] == 1]
train_negative = train_df[train_df["label"] == 2]

train_df = pd.concat([
    train_neutral.sample(87),
    train_positive,
    train_negative.sample(87)])

In [87]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'

bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [88]:
text_test = ['Sveiki! Informējam, ka saistībā ar tehniskiem uzlabojumiem šonakt laika posmā no plkst. 2.00 līdz 4.00 BITES tīklā īslaicīgi – aptuveni 3 min – nebūs pieejams internets. Ja tas nav pieejams ilgāku laiku, aicinām restartēt ierīces. Atvainojamies par sagādātajām neērtībām!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [  101 77439 20897   106 43964 19341 19542 10147   117 10730 13410 10562]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [89]:
bert_model = hub.KerasLayer(encoder_url)

In [90]:
bert_results = bert_model(text_preprocessed)

print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[ 0.23613805 -0.044425    0.23505512 -0.1577017  -0.03203797  0.24510026
  0.09333638  0.13524106 -0.20804936  0.1443368   0.08714204 -0.15120126]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[-0.27231994  0.03256027 -0.04421553 ...  0.09869209  0.35841864
  -0.14795616]
 [-0.6611898   0.03879993  0.28476664 ... -0.34495085  0.92087716
  -0.09457584]
 [-0.3153755  -0.16312945  0.38296917 ... -0.22015077  0.79261357
  -0.6949447 ]
 ...
 [-0.68508154  0.30632207  0.47708502 ... -0.11326861  0.65564984
  -0.10139303]
 [-0.2931506  -0.7152828  -0.3765017  ... -1.0267605   1.0475307
  -0.09604091]
 [-0.02384593 -1.0335579  -0.11690581 ...  0.7291889   0.9226878
  -0.9616023 ]]


In [115]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='message_lowercase')
    preprocessing_layer = hub.KerasLayer(preprocess_url, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [116]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.4821939]], shape=(1, 1), dtype=float32)


In [117]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [118]:
epochs = 5
steps_per_epoch = 3
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [119]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [121]:
print(f'Training model with {encoder_url}')
history = classifier_model.fit(x=csv_features,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4
Epoch 1/5
2/2 [==============================] - 968s 127s/step - loss: 0.5580 - binary_accuracy: 0.3403
Epoch 2/5
2/2 [==============================] - 962s 132s/step - loss: 0.5142 - binary_accuracy: 0.2941
Epoch 3/5
2/2 [==============================] - 950s 130s/step - loss: 0.4296 - binary_accuracy: 0.3298
Epoch 4/5
2/2 [==============================] - 973s 130s/step - loss: 0.3580 - binary_accuracy: 0.4433
Epoch 5/5
2/2 [==============================] - 941s 131s/step - loss: 0.2149 - binary_accuracy: 0.4937


In [122]:
loss, accuracy = classifier_model.evaluate(csv_features)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

2/2 [==============================] - 260s 43s/step - loss: 0.0684 - binary_accuracy: 0.5357
Loss: 0.06841637194156647
Accuracy: 0.5357142686843872
